### Data Augumentation - Augraphy

> **⚠️실행 전 주의 사항⚠️**
- 증강된 이미지를 저장할 경로를 설정합니다.
- 증강된 이미지 생성을 위해 함수 `augment_images()`의 `매개변수(s_index, e_index, n_iteration)`를 지정합니다.
    - `s_index`: 이미지 증강을 실행할 원본 이미지 시작 인덱스 (0부터 시작)
    - `e_index`: 이미지 증강을 수행할 원본 이미지 마지막 인덱스
    - `n_interation`: 이미지 증강 수행 횟수 (무작위성이 있음)
- 증강된 이미지 레이블을 저장할 경로를 설정합니다. 
- 증강된 이미지 레이블을 저장할 파일명(*.csv)을 설정합니다.

#### 1) 라이브러리 다운로드 및 임포트

In [1]:
# !pip install augraphy
# !pip install albumentations

In [2]:
import os

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# 원본 이미지 경로 탐색 및 파일명 추출
fpath_load = "../data/train/"
fname_list = os.listdir(fpath_load)
fname_list.sort()
print("원본 이미지 개수: ", len(fname_list))

# 증강 이미지 저장 경로 설정
fpath_save = "../data/train_augmented_tmp/"
# fpath_save = "../data/train_augmented/"
if not os.path.exists(fpath_save):
    os.makedirs(fpath_save)

원본 이미지 개수:  1570


#### 2) Augraphy with Default Pipeline

In [4]:
from augraphy import *


# 이미지 증강
def augment_images_default(s_index=0, e_index=1569, n_iteration=10):
    # Make Pipeline for Augraphy
    pipeline = default_augraphy_pipeline()

    # Make Batches of Images
    image_list = []
    for i in range(s_index, e_index + 1):
        # 원본 이미지 파일명 리스트 생성
        image_list.append(cv.imread(fpath_load + fname_list[i]))
        # 원본 이미지 저장
        cv.imwrite(fpath_save + fname_list[i], image_list[i - s_index])
        print(f"원본 이미지 저장: {fpath_save}{fname_list[i]}")

    # Augument Images
    for i in range(1, n_iteration + 1):
        # 이미지 증강
        image_augmented_list = pipeline(image_list)

        for j, image in enumerate(image_augmented_list):
            # 증강 이미지 저장
            fname = fname_list[s_index + j].split(".")[0]
            fname_save = f"{fpath_save}{fname}_{i}.jpg"
            cv.imwrite(fname_save, image)
            print(f"증강 이미지 저장: {fname_save}")

    return image_augmented_list

In [5]:
# # 이미지 증강 실행
# augment_images_default(s_index=0, e_index=4, n_iteration=10)

#### 데이터 레이블 저장

In [6]:
def augment_labels():

    fname_augmented_path = fpath_save
    fname_augmented_list = os.listdir(fname_augmented_path)
    fname_augmented_list.sort()

    df_train_augmented = pd.DataFrame(columns=[["ID", "target"]])
    df_train_augmented["ID"] = fname_augmented_list
    # print(df_train_augmented)

    df_train = pd.read_csv("../data/train.csv")
    df_train = df_train.set_index("ID").T.to_dict()

    for i in range(len(df_train_augmented)):
        df_train_augmented_ID = df_train_augmented.iloc[i]["ID"]
        df_train_augmented_ID = (
            df_train_augmented_ID.replace("_", ".").split(".")[0] + ".jpg"
        )

        for key, value in df_train.items():
            if df_train_augmented_ID == key:
                df_train_augmented.at[i, "target"] = value["target"]

    print("원본 + 증강 이미지 개수: ", df_train_augmented.shape[0])
    print("결측치")
    print(df_train_augmented.isnull().sum())

    df_train_augmented.to_csv("../data/train_augmented_tmp.csv", index=False)
    # df_train_augmented.to_csv("../data/train_augmented.csv", index=False)

In [7]:
# augment_labels()

#### 3) Augraphy - User Defined Pipeline

##### 하나의 Augraphy User Defined Pipeline

In [8]:
from augraphy import *
from MyAugraphyPipeline import *


# 이미지 증강 함수
# - 사용자 정의 파이프라인을 10번 수행 (매개변수로 지정한 범위 내에서 랜덤하게 수행)
def augment_images_with_my_pipeline_one(s_index=0, e_index=1569, n_iteration=10):
    # Make Pipeline for Augraphy
    pipeline = my_augraphy_pipeline()

    # Make Batches of Images
    image_list = []
    for i in range(s_index, e_index + 1):
        # 원본 이미지 파일명 리스트 생성
        image_list.append(fpath_load + fname_list[i])

        # 원본 이미지 저장
        cv.imwrite(fpath_save + fname_list[i], image_list[i - s_index])
        print(f"원본 이미지 저장: {fpath_save}{fname_list[i]}")

    # Augument Images
    for i in range(1, n_iteration + 1):
        # 이미지 증강
        image_augmented_list = pipeline(image_list)

        for j, image in enumerate(image_augmented_list):
            # 증강 이미지 저장
            fname = fname_list[s_index + j].split(".")[0]
            fname_save = f"{fpath_save}{fname}_augmented_{i}.jpg"
            cv.imwrite(fname_save, image)
            print(f"증강 이미지 저장: {fname_save}")

In [9]:
# augment_images_with_my_pipeline_one(s_index=0, e_index=1, n_iteration=10)

##### 다수의 Augraphy User Defined Pipeline

In [10]:
from augraphy import *
from augraphy.default.pipeline import *
from MyAugraphyPipeline import *


# 이미지 증강 함수
# - 사용자 정의 파이프라인 10개를 1번 돌림
def augment_images_with_my_pipeline_list(s_index=0, e_index=1569, n_iteration=10):
    # Make Pipelines for Augraphy
    pipeline_list = [
        # 미리 정의된 함수
        # pipeline_archetype1(),
        # pipeline_archetype2(),
        # pipeline_archetype3(),
        # pipeline_archetype4(),
        # pipeline_archetype5(),
        # pipeline_archetype6(),
        # pipeline_archetype7(),
        # # pipeline_archetype8(), # 오류
        # pipeline_archetype9(),
        # pipeline_archetype10(),
        # 사용자 정의 함수 (*현재 my_augraphy_pipeline1만 구현)
        my_augraphy_pipeline1(),
        # my_augraphy_pipeline2(),
        # my_augraphy_pipeline3(),
        # my_augraphy_pipeline4(),
        # my_augraphy_pipeline5(),
    ]

    image_list = []
    for i in range(s_index, e_index + 1):
        # 원본 이미지 파일명 리스트 생성
        image_list.append(cv.imread(fpath_load + fname_list[i]))
        # 원본 이미지 저장
        cv.imwrite(fpath_save + fname_list[i], image_list[i - s_index])
        print(f"원본 이미지 저장: {fpath_save}{fname_list[i]}")

    # Make Batches of Images
    for n in range(n_iteration):
        # Augument Images
        ## 첫번째 파이프라인 실행 → 두번째 파이프라인 실행 → ...
        for i in range(len(pipeline_list)):
            # 사용자 정의 파이프라인을 통한 이미지 증강
            globals()[f"pipeline{i}"] = pipeline_list[i]
            globals()[f"image_augmented_list{i}"] = globals()[f"pipeline{i}"](
                image_list
            )

            # 증강 이미지 저장
            for j, image in enumerate(globals()[f"image_augmented_list{i}"]):
                fname = fname_list[s_index + j].split(".")[0]
                fname_save = f"{fpath_save}{fname}_augmented_n{n}-p{i}-{j}.jpg"
                cv.imwrite(fname_save, image)
                print(f"증강 이미지 저장: {fname_save}")

            # # 원본 이미지 출력
            # image_rows = 1
            # image_columns = len(image_list)
            # plt.figure(figsize=(25, 25))
            # for k in range(image_rows * image_columns):
            #     image_index = k + 1
            #     image_title = "image {}".format(image_index)

            #     plt.subplot(image_rows, image_columns, image_index)
            #     plt.title(image_title)
            #     plt.xticks([])  # x = None
            #     plt.yticks([])  # y = None
            #     plt.imshow(image_list[k])

            # 증강 이미지 출력
            image_rows = 1
            image_columns = len(image_list)
            plt.figure(figsize=(25, 25))
            for k in range(image_rows * image_columns):
                image_index = k + 1
                image_title = "image_augmented {}".format(image_index)

                plt.subplot(image_rows, image_columns, image_index)
                plt.title(image_title)
                plt.xticks([])  # x = None
                plt.yticks([])  # y = None
                plt.imshow(globals()[f"image_augmented_list{i}"][k])

            plt.show()

In [11]:
# augment_images_with_my_pipeline_list(s_index=0, e_index=4, n_iteration=10)

##### 다수의 Augrapy User Defined Pipeline + Albumatation 적용

In [12]:
# # 이미지 크기 확인
# for i in range(10):
#     image = cv.imread(fpath_load + fname_list[i])

#     height, width, channels = image.shape
#     print(f"Index{i} Width: {width}, Height: {height}, Channels: {channels}")

In [13]:
df = pd.read_csv("../data/train.csv")
target_values = [3, 4, 7, 14]
target_filtered_list = df[df["target"].isin(target_values)]["ID"].tolist()

print(target_filtered_list)

['009235e4c9c07af5.jpg', '012913977fd1d980.jpg', '0250ee8107091ade.jpg', '02dad82a9420ae86.jpg', '03084c1b03921a99.jpg', '04320d1d34f005ca.jpg', '055b74c6c5f5d86b.jpg', '05c30f3c5be4bdcb.jpg', '067f25e0435ab78b.jpg', '06c8db1a419a2a52.jpg', '06e3ae9dce375043.jpg', '0808c7453316041c.jpg', '08e735737fcb818d.jpg', '09a930520afa0ab9.jpg', '0a9843bf4ec022f3.jpg', '0d6a14437ad1a20e.jpg', '0e15e264405bd12a.jpg', '0f3ac436363ae727.jpg', '1258871e4d7347be.jpg', '134a6165b1e7198b.jpg', '1495a1e3d1599fe1.jpg', '15738105f616bea2.jpg', '159f76320c144a6a.jpg', '15f0ed8589380d8f.jpg', '16176cca20eed3c8.jpg', '1674ad0591a3014a.jpg', '18a13df74db514bf.jpg', '1a5b5d5ceed7b946.jpg', '1aa909832b5c4542.jpg', '1ab6b4d6b1ebe5dd.jpg', '1af85d095317707e.jpg', '1bb0c218c35112e9.jpg', '1c893f02de43b62a.jpg', '1cc56bce8c1d4a39.jpg', '1de204716ce483ca.jpg', '1df37977197865cb.jpg', '1e5bf6fe5e8686d6.jpg', '1ec14a14bbe633db.jpg', '1fbda5ffdc6abf49.jpg', '1fca58126b4d1128.jpg', '210e5e6b28b73a4b.jpg', '22819e513decca

In [14]:
from augraphy import *
from augraphy.default.pipeline import *
from MyAugraphyPipeline import *
import albumentations as A


# 이미지 증강 함수
# - 사용자 정의 파이프라인 10개를 1번 돌림
def augment_images_with_albumentations(s_index=0, e_index=1569, n_iteration=10):
    # Make Pipelines for Augraphy
    pipeline_list = [
        # 미리 정의된 함수
        # pipeline_archetype1(),
        # pipeline_archetype2(),
        # pipeline_archetype3(),
        # pipeline_archetype4(),
        # pipeline_archetype5(),
        # pipeline_archetype6(),
        # pipeline_archetype7(),
        # # pipeline_archetype8(), # 오류
        # pipeline_archetype9(),
        # pipeline_archetype10(),
        # 사용자 정의 함수 (*현재 my_augraphy_pipeline1만 구현)
        my_augraphy_pipeline1(),
        # my_augraphy_pipeline2(),
        # my_augraphy_pipeline3(),
        # my_augraphy_pipeline4(),
        # my_augraphy_pipeline5(),
    ]

    # albumentations 증강 파이프라인 정의
    transform = A.Compose(
        [
            # 회전 변환
            A.Rotate(
                limit=360, border_mode=cv.BORDER_CONSTANT, value=(255, 255, 255), p=0.5
            ),  # 360도 이내 회전, 빈 공간을 흰색으로 채움, 항상 적용
            # 크롭 변환 (중앙 크롭)
            # A.CenterCrop(height=224, width=224, p=0.5),  # 중앙 크롭, 항상 적용
            # 추가적인 변환 (선택적)
            A.RandomBrightnessContrast(p=0.5),  # 밝기/대비 조정
            A.HorizontalFlip(p=0.5),  # 좌우 반전
            A.ShiftScaleRotate(
                shift_limit=0.1,
                scale_limit=0.1,
                rotate_limit=45,
                border_mode=cv.BORDER_CONSTANT,  # 빈 공간을 고정된 색상으로 채움
                value=(255, 255, 255),  # 빈 공간을 흰색으로 채움
                p=0.25,  # 항상 적용
            ),  # 이동, 확대/축소 및 추가 회전
        ]
    )

    image_list = []
    for i in range(s_index, e_index + 1):
        # 이미지 리스트 생성
        image_list.append(cv.imread(fpath_load + target_filtered_list[i]))
        # 이미지 저장
        cv.imwrite(fpath_save + target_filtered_list[i], image_list[i])
        print(f"원본 이미지 저장: {fpath_save}{target_filtered_list[i]}")

    # Make Batches of Images
    for n in range(n_iteration):
        for i, pipeline in enumerate(pipeline_list):
            # Augraphy 증강 적용
            image_augmented_list = pipeline(image_list)

            # Almumentations 증강 적용 및 이미지 저장
            image_transformed_list = []
            for j, image in enumerate(image_augmented_list):
                # Almumentations 증강 적용
                transformed = transform(image=image)
                image_transformed = transformed["image"]  # 이미지 데이터를 추출
                image_transformed_rgb = cv.cvtColor(image_transformed, cv.COLOR_BGR2RGB)
                image_transformed_list.append(image_transformed_rgb)

                # 이미지 저장
                fname = target_filtered_list[s_index + j].split(".")[0]
                fname_save = f"{fpath_save}{fname}_transformed_n{n}_p{i}.jpg"
                cv.imwrite(fname_save, image_transformed_rgb)
                print(f"증강 이미지 저장: {fname_save}")

            # # 원본 이미지 출력
            # image_rows = 1
            # image_columns = len(image_list)
            # plt.figure(figsize=(25, 25))
            # for k in range(image_rows * image_columns):
            #     image_index = k + 1
            #     image_title = "image {}".format(image_index)

            #     plt.subplot(image_rows, image_columns, image_index)
            #     plt.title(image_title)
            #     plt.xticks([])  # x = None
            #     plt.yticks([])  # y = None
            #     plt.imshow(image_list[k])
            #     plt.show()

            # # 증강 이미지 출력
            # image_rows = 1
            # image_columns = len(image_transformed_list)
            # plt.figure(figsize=(25, 25))
            # for k in range(image_rows * image_columns):
            #     image_index = k + 1
            #     image_title = "image_transformed {}".format(image_index)

            #     plt.subplot(image_rows, image_columns, image_index)
            #     plt.title(image_title)
            #     plt.xticks([])  # x = None
            #     plt.yticks([])  # y = None
            #     plt.imshow(image_transformed_list[k])
            #     plt.show()

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.13 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [15]:
augment_images_with_albumentations(s_index=0, e_index=1, n_iteration=5)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


원본 이미지 저장: ../data/train_augmented_tmp/009235e4c9c07af5.jpg
원본 이미지 저장: ../data/train_augmented_tmp/012913977fd1d980.jpg
증강 이미지 저장: ../data/train_augmented_tmp/009235e4c9c07af5_transformed_n0_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/012913977fd1d980_transformed_n0_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/009235e4c9c07af5_transformed_n1_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/012913977fd1d980_transformed_n1_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/009235e4c9c07af5_transformed_n2_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/012913977fd1d980_transformed_n2_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/009235e4c9c07af5_transformed_n3_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/012913977fd1d980_transformed_n3_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/009235e4c9c07af5_transformed_n4_p0.jpg
증강 이미지 저장: ../data/train_augmented_tmp/012913977fd1d980_transformed_n4_p0.jpg


In [16]:
augment_labels()

원본 + 증강 이미지 개수:  12
결측치
ID        0
target    0
dtype: int64
